In [1]:
pip install torchio


  Using cached Deprecated-1.2.18-py2.py3-none-any.whl.metadata (5.7 kB)
  Using cached click-8.1.8-py3-none-any.whl.metadata (2.3 kB)
  Using cached shellingham-1.5.4-py2.py3-none-any.whl.metadata (3.5 kB)
Using cached Deprecated-1.2.18-py2.py3-none-any.whl (10.0 kB)
   ---------------------------------------- 0.0/40.9 MB ? eta -:--:--
   ------- -------------------------------- 8.1/40.9 MB 42.1 MB/s eta 0:00:01
   ---------------- ----------------------- 17.0/40.9 MB 43.0 MB/s eta 0:00:01
   -------------------------- ------------- 27.3/40.9 MB 44.3 MB/s eta 0:00:01
   --------------------------------- ------ 34.3/40.9 MB 42.0 MB/s eta 0:00:01
   ---------------------------------------  40.9/40.9 MB 42.6 MB/s eta 0:00:01
   ---------------------------------------- 40.9/40.9 MB 38.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/18.8 MB ? eta -:--:--
   --------------- ------------------------ 7.1/18.8 MB 36.4 MB/s eta 0:00:01
   --------------------------------- ----

Augmentation


In [2]:
import torchio as tio

# TorchIO Transform Pipeline
transform = tio.Compose([
    tio.RandomFlip(axes=(0, 1, 2), flip_probability=0.5),         # Random flip on all axes
    tio.RandomAffine(scales=(0.9, 1.1), degrees=10, translation=5),  # Small rotation, scaling, shift
    tio.RandomNoise(mean=0, std=0.1),                             # Add Gaussian noise
    tio.RandomBiasField(coefficients=0.5),                        # MRI-specific artifact
    tio.RandomGamma(log_gamma=(-0.3, 0.3)),                       # Gamma contrast adjustment
])


c:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\gpu_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
!pip install torchio

In [2]:
import os
import numpy as np
import nibabel as nib
import torch
from torch.utils.data import Dataset, DataLoader
import torchio as tio

# TorchIO Transform Pipeline
transform = tio.Compose([
    tio.RandomFlip(axes=(0, 1, 2), flip_probability=0.5),         # Random flip on all axes
    tio.RandomAffine(scales=(0.9, 1.1), degrees=10, translation=5),  # Small rotation, scaling, shift
    tio.RandomNoise(mean=0, std=0.1),                             # Add Gaussian noise
    tio.RandomBiasField(coefficients=0.5),                        # MRI-specific artifact
    tio.RandomGamma(log_gamma=(-0.3, 0.3)),                       # Gamma contrast adjustment
])


class MRIDataset(Dataset):
    def __init__(self, pre_paths, ce_paths, seg_paths, transform=None):
        self.pre_paths = pre_paths
        self.ce_paths = ce_paths
        self.seg_paths = seg_paths
        self.transform = transform

    def __len__(self):
        return len(self.pre_paths)

    def __getitem__(self, idx):
        pre = nib.load(self.pre_paths[idx]).get_fdata().astype(np.float32)
        ce = nib.load(self.ce_paths[idx]).get_fdata().astype(np.float32)
        seg = nib.load(self.seg_paths[idx]).get_fdata().astype(np.float32)

        # Normalize volumes
        pre = (pre - pre.min()) / (pre.max() - pre.min() + 1e-8)
        ce = (ce - ce.min()) / (ce.max() - ce.min() + 1e-8)

        # Add channel dimension
        pre = np.expand_dims(pre, axis=0)
        ce = np.expand_dims(ce, axis=0)
        seg = np.expand_dims(seg, axis=0)

        subject = tio.Subject(
            pre=tio.ScalarImage(tensor=pre),
            ce=tio.ScalarImage(tensor=ce),
            seg=tio.LabelMap(tensor=seg)
        )

        if self.transform:
            subject = self.transform(subject)

        pre = subject['pre'].data
        ce = subject['ce'].data
        seg = subject['seg'].data

        return pre, ce, seg


c:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\gpu_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import sys
print(sys.executable)


c:\Users\OMEN\Desktop\MRI_Synthesis_Project\MRI_SYNTHESIS_PROJECT\gpu_env\Scripts\python.exe
